# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

In [ ]:
## Question 1
This question will walk you through creating your own `kmeans` function.

In [2]:
#### a) What are the steps of `kmeans`?

In [ ]:
ANSWER For Question 1a:

1. Initialization — choose K initial centroids.

2. Assignment — assign each data point to the nearest centroid.

3. Update — recompute centroids from assigned points (two ways: loop or vectorized/accumarray — the latter is much faster).

4. Convergence / stopping check — stop if centroids move less than tolerance or max iterations reached.

#### b) Create the builder function for step 1.

In [ ]:
# 1) Initialization
init_centroids <- function(X, K, init = "sample", seed = NULL) {
  if (!is.null(seed)) set.seed(seed)
  n <- nrow(X); d <- ncol(X)
  if (tolower(init) == "sample") {
    idx <- sample(n, K)
    return(X[idx, , drop = FALSE])
  }
  if (tolower(init) == "kmeans++") {
    C <- matrix(0, nrow = K, ncol = d)
    C[1, ] <- X[sample(n, 1), ]
    for (k in 2:K) {
      xr2 <- rowSums(X^2)
      cprev2 <- rowSums(C[1:(k-1), , drop = FALSE]^2)
      Dall <- outer(xr2, cprev2, "+") - 2 * (X %*% t(C[1:(k-1), , drop = FALSE]))
      minD <- apply(Dall, 1, min)
      if (all(minD == 0)) {
        p <- rep(1/n, n)
      } else {
        minD[minD < 0] <- 0
        p <- minD / sum(minD)
      }
      nexti <- which.max(runif(1) <= cumsum(p))
      if (length(nexti) == 0) nexti <- sample(n, 1)
      C[k, ] <- X[nexti, ]
    }
    return(C)
  }
  stop("Unknown init method")
}

#### c) Create the builder function for step 2.

In [ ]:
# 2) Assignment
assign_labels <- function(X, C) {
  xr2 <- rowSums(X^2)
  cr2 <- rowSums(C^2)
  D <- outer(xr2, cr2, "+") - 2 * (X %*% t(C))
  labels <- max.col(-D)
  labels
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [ ]:
# 3a) Update - vectorized (preferred)
update_centroids_vectorized <- function(X, labels, K) {
  d <- ncol(X)
  sums <- rowsum(X, labels)
  counts <- as.vector(table(factor(labels, levels = 1:K)))
  C_new <- matrix(NA_real_, nrow = K, ncol = d)
  nonempty <- which(counts > 0)
  if (length(nonempty) > 0) C_new[nonempty, ] <- sums / counts[nonempty]
  empty <- which(counts == 0)
  if (length(empty) > 0) {
    n <- nrow(X)
    for (e in empty) C_new[e, ] <- X[sample(n, 1), ]
  }
  C_new
}

# 3b) Update - loop (alternative)
update_centroids_loop <- function(X, labels, K) {
  d <- ncol(X); n <- nrow(X)
  C_new <- matrix(NA_real_, nrow = K, ncol = d)
  for (k in seq_len(K)) {
    idx <- which(labels == k)
    if (length(idx) == 0) {
      C_new[k, ] <- X[sample(n, 1), ]
    } else {
      C_new[k, ] <- colMeans(X[idx, , drop = FALSE])
    }
  }
  C_new
}

#### e) Create the builder function for step 4.

In [ ]:
# 4) Convergence / stopping check
convergence_check <- function(C_old, C_new, tol = 1e-6, labels_old = NULL, labels_new = NULL) {
  moves <- sqrt(rowSums((C_old - C_new)^2))
  moved_less_than_tol <- max(moves, na.rm = TRUE) < tol
  labels_unchanged <- !is.null(labels_old) && !is.null(labels_new) && identical(labels_old, labels_new)
  list(converged = moved_less_than_tol || labels_unchanged, moves = moves)
}

#### f) Combine them all into your own `kmeans` function.

In [ ]:
# Composed kmeans
my_kmeans <- function(X, K, max.iter = 100, tol = 1e-6,
                      init = "sample", seed = NULL,
                      update_method = c("vectorized", "loop"),
                      verbose = FALSE) {
  if (!is.matrix(X)) X <- as.matrix(X)
  n <- nrow(X); d <- ncol(X)
  if (!is.null(seed)) set.seed(seed)
  if (K > n) stop("K must be <= number of data points")
  update_method <- match.arg(update_method)
  C <- init_centroids(X, K, init = init, seed = seed)
  labels <- integer(n)
  history <- list(C = list(C))
  it <- 0
  converged <- FALSE
  while (it < max.iter && !converged) {
    it <- it + 1
    labels_old <- labels
    labels <- assign_labels(X, C)
    if (update_method == "vectorized") {
      C_new <- update_centroids_vectorized(X, labels, K)
    } else {
      C_new <- update_centroids_loop(X, labels, K)
    }
    chk <- convergence_check(C, C_new, tol = tol, labels_old = labels_old, labels_new = labels)
    converged <- chk$converged
    C <- C_new
    history$C[[length(history$C) + 1]] <- C
    if (verbose) cat(sprintf("iter %d max move = %g\n", it, max(chk$moves, na.rm = TRUE)))
  }
  list(labels = labels, means = C, iter = it, history = history)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [ ]:
Answer under b and c where i do this

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [5]:
# run_my_kmeans.R
# Full R implementation (four parts) and run on voltages_df.csv

# 1) Initialization
init_centroids <- function(X, K, init = "sample", seed = NULL) {
  if (!is.null(seed)) set.seed(seed)
  n <- nrow(X); d <- ncol(X)
  if (tolower(init) == "sample") {
    idx <- sample(n, K)
    return(X[idx, , drop = FALSE])
  }
  if (tolower(init) == "kmeans++") {
    C <- matrix(0, nrow = K, ncol = d)
    C[1, ] <- X[sample(n, 1), ]
    for (k in 2:K) {
      xr2 <- rowSums(X^2)
      cprev2 <- rowSums(C[1:(k-1), , drop = FALSE]^2)
      Dall <- outer(xr2, cprev2, "+") - 2 * (X %*% t(C[1:(k-1), , drop = FALSE]))
      minD <- apply(Dall, 1, min)
      if (all(minD == 0)) {
        p <- rep(1/n, n)
      } else {
        minD[minD < 0] <- 0
        p <- minD / sum(minD)
      }
      nexti <- which.max(runif(1) <= cumsum(p))
      if (length(nexti) == 0) nexti <- sample(n, 1)
      C[k, ] <- X[nexti, ]
    }
    return(C)
  }
  stop("Unknown init method")
}

# 2) Assignment
assign_labels <- function(X, C) {
  xr2 <- rowSums(X^2)
  cr2 <- rowSums(C^2)
  D <- outer(xr2, cr2, "+") - 2 * (X %*% t(C))
  labels <- max.col(-D)
  labels
}

# 3a) Update - vectorized (preferred)
update_centroids_vectorized <- function(X, labels, K) {
  d <- ncol(X)
  sums <- rowsum(X, labels)
  counts <- as.vector(table(factor(labels, levels = 1:K)))
  C_new <- matrix(NA_real_, nrow = K, ncol = d)
  nonempty <- which(counts > 0)
  if (length(nonempty) > 0) C_new[nonempty, ] <- sums / counts[nonempty]
  empty <- which(counts == 0)
  if (length(empty) > 0) {
    n <- nrow(X)
    for (e in empty) C_new[e, ] <- X[sample(n, 1), ]
  }
  C_new
}

# 3b) Update - loop (alternative)
update_centroids_loop <- function(X, labels, K) {
  d <- ncol(X); n <- nrow(X)
  C_new <- matrix(NA_real_, nrow = K, ncol = d)
  for (k in seq_len(K)) {
    idx <- which(labels == k)
    if (length(idx) == 0) {
      C_new[k, ] <- X[sample(n, 1), ]
    } else {
      C_new[k, ] <- colMeans(X[idx, , drop = FALSE])
    }
  }
  C_new
}

# 4) Convergence / stopping check
convergence_check <- function(C_old, C_new, tol = 1e-6, labels_old = NULL, labels_new = NULL) {
  moves <- sqrt(rowSums((C_old - C_new)^2))
  moved_less_than_tol <- max(moves, na.rm = TRUE) < tol
  labels_unchanged <- !is.null(labels_old) && !is.null(labels_new) && identical(labels_old, labels_new)
  list(converged = moved_less_than_tol || labels_unchanged, moves = moves)
}

# Composed kmeans
my_kmeans <- function(X, K, max.iter = 100, tol = 1e-6,
                      init = "sample", seed = NULL,
                      update_method = c("vectorized", "loop"),
                      verbose = FALSE) {
  if (!is.matrix(X)) X <- as.matrix(X)
  n <- nrow(X); d <- ncol(X)
  if (!is.null(seed)) set.seed(seed)
  if (K > n) stop("K must be <= number of data points")
  update_method <- match.arg(update_method)
  C <- init_centroids(X, K, init = init, seed = seed)
  labels <- integer(n)
  history <- list(C = list(C))
  it <- 0
  converged <- FALSE
  while (it < max.iter && !converged) {
    it <- it + 1
    labels_old <- labels
    labels <- assign_labels(X, C)
    if (update_method == "vectorized") {
      C_new <- update_centroids_vectorized(X, labels, K)
    } else {
      C_new <- update_centroids_loop(X, labels, K)
    }
    chk <- convergence_check(C, C_new, tol = tol, labels_old = labels_old, labels_new = labels)
    converged <- chk$converged
    C <- C_new
    history$C[[length(history$C) + 1]] <- C
    if (verbose) cat(sprintf("iter %d max move = %g\n", it, max(chk$moves, na.rm = TRUE)))
  }
  list(labels = labels, means = C, iter = it, history = history)
}

# ------------------------
# Run on voltages_df.csv
# ------------------------
if (!file.exists("voltages_df.csv")) stop('File "voltages_df.csv" not found in working directory.')
voltages_df <- read.csv("voltages_df.csv", stringsAsFactors = FALSE)
X <- as.matrix(voltages_df)
set.seed(1)
results <- my_kmeans(X, K = 3, init = "sample", seed = 1, verbose = FALSE)

cat("=== Results from my_kmeans() ===\n")
cat("results$labels:\n"); print(results$labels)
cat("results$means:\n");  print(results$means)

=== Results from my_kmeans() ===
results$labels:
  [1] 1 2 2 2 2 2 3 3 2 1 1 1 3 1 2 1 1 1 1 1 2 2 2 2 2 1 1 1 3 3 3 3 2 3 3 2 2
 [38] 1 3 2 1 1 1 3 2 3 3 2 3 3 2 3 1 1 3 1 2 1 3 2 1 1 2 1 3 3 2 3 1 3 1 2 3 3
 [75] 2 2 1 3 3 3 3 3 2 3 3 2 2 2 1 3 3 3 2 3 2 2 3 2 2 3 3 3 2 3 1 1 3 1 1 2 1
[112] 1 2 3 1 3 1 1 2 2 2 2 1 1 3 2 2 3 1 3 2 1 2 3 1 2 2 2 1 3 1 2 2 3 1 1 2 1
[149] 1 3 1 3 2 2 3 1 3 3 2 1 3 3 1 2 1 3 2 1 2 1 1 3 2 1 3 3 2 3 1 2 3 2 3 1 3
[186] 2 2 1 2 1 1 2 1 1 3 3 1 2 1 1 3 1 3 3 2 1 2 3 2 1 2 3 1 3 3 2 3 3 2 2 2 2
[223] 2 2 2 3 3 3 3 1 2 1 3 2 2 3 3 1 1 3 3 1 3 3 2 3 1 3 3 2 3 2 1 3 2 3 3 1 1
[260] 1 2 2 3 1 2 2 2 3 3 3 1 2 2 2 3 1 3 3 2 2 3 1 3 1 1 2 3 2 2 3 3 1 3 3 1 2
[297] 1 3 3 2 2 3 1 1 1 3 1 2 2 2 1 1 1 1 1 1 2 1 2 1 1 3 2 3 1 1 1 1 2 1 2 1 3
[334] 1 2 2 3 3 1 2 3 2 3 3 1 3 2 3 1 1 3 2 2 2 1 3 2 1 3 3 2 2 2 2 1 3 1 2 1 1
[371] 1 1 1 2 2 3 3 1 1 1 1 3 3 1 1 3 3 2 2 3 1 2 2 2 2 3 2 2 3 1 1 1 1 2 2 1 1
[408] 3 3 3 2 3 3 1 2 2 2 3 2 2 2 1 1 2 1 3 2 2 2 2 3 3 2 3 1 2 1 1 1 3

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [10]:
# part2_base_kmeans.R

# Read data
voltages_df <- read.csv("voltages_df.csv", stringsAsFactors = FALSE)
X <- as.matrix(voltages_df)

# Run base kmeans with K = 3
set.seed(1)
results <- kmeans(X, centers = 3, nstart = 1)  # results contains $centers and $cluster

cat("=== Part 2: base::kmeans() ===\n")
cat("results$centers:\n"); print(results$centers)
cat("results$cluster:\n"); print(results$cluster)

=== Part 2: base::kmeans() ===
results$centers:
         X0 X1.00401606425703 X2.00803212851406 X3.01204819277108
1 -1.031463         0.9381238         0.7619864         0.3631543
2 -1.031463         1.2439759         1.0924697         0.9004440
3 -1.031463         1.3093239         1.1616772         0.9787498
  X4.01606425702811 X5.02008032128514 X6.02409638554217 X7.0281124497992
1        -1.1179412         -1.051145        -0.9766807       -0.8694758
2         0.3011754         -1.159714        -1.1098127       -1.0685484
3         0.6481497         -1.168610        -1.1196122       -1.0590962
  X8.03212851405623 X9.03614457831325 X10.0401606425703 X11.0441767068273
1        -0.6892375        -0.5661321        -0.2497152         0.6027358
2        -1.0338649        -1.0022396        -0.9699741        -0.9343697
3        -0.9943176        -0.9237437        -0.8457536        -0.7572129
  X12.0481927710843 X13.0522088353414 X14.0562248995984 X15.0602409638554
1         0.6960355       

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

In [ ]:
Answer:
Yes, my labels and means match. The results$labels from my my_kmeans() and results$cluster 
from base kmeans() are element-wise identical and the results$means matrix equals results$centers.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

In [ ]:
Answer:
Using a for-loop to assign clusters in k-means is the straightforward, “textbook” approach: 
for each data point you loop over it, compute its distance to every centroid, pick the centroid 
with the smallest distance, and store that centroid’s index as the point’s cluster label. Concretely, 
inside the outer loop you either loop over centroids to compute distances or compute distances in one call 
and then use argmin to select the nearest centroid; after the loop over all points you have an assignment vector 
and can recompute centroids by averaging the points assigned to each cluster. This approach is easy to understand and 
simple to write, but it pays the overhead of interpreted loop iterations (one iteration per point) and repeated 
distance computations inside the interpreter.


#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

In [ ]:
Answer:
Vectorizing the assignment step replaces explicit elementwise loops with matrix operations that compute distances 
for many points at once. A common method uses the identity |x−μ|² = |x|² + |μ|² − 2 x·μ: form a matrix of dot products X * M prime,
add the squared norms of points and centroids (broadcasted) to produce an 𝑁×𝐾N×K matrix of squared distances (N points, K centroids), 
then take argmin across the centroid dimension to get each point’s nearest centroid in one call. The cluster membership can be represented 
as an 𝑁×𝐾N×K one-hot matrix (logical mask) produced without loops, and new centroids computed with a handful of matrix ops 
(e.g. sum of points per cluster via transposed mask times X, divided by cluster counts). This uses linear-algebra primitives 
instead of per-point work and avoids interpreter overhead.

#### c) State which (for loops or vectorizing) is more efficient and why.

In [ ]:
Answer:
Vectorizing is usually faster because it lets MATLAB do the heavy math all at once in compiled code, 
instead of running many slow interpreter-level loops caused by for loops.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

In [ ]:
Answer:
K-means can fail if clusters are not roughly spherical, equal in size and density, or when clusters overlap 
heavily or have different covariance shapes. This is due to k-means’ core assumption, where it models clusters 
by minimizing within-cluster variance, effectively assuming each cluster is isotropic (spherical) and of comparable scale. 
This means that k-means will fail and split elongated or ellipsoidal clusters, merge small dense clusters with nearby large emptier ones, and
be misled by different variances or outliers.

## Question 5
#### What assumption do Guassian mixture models make?

In [ ]:
Answer:
Gaussian mixture models (GMMs) assume the data were generated by a finite mixture of Gaussian, or normal, distributions: 
each observation is drawn from one of K Gaussian components, where each component has its own mean and covariance and mixture weight. 
The model therefore assumes that 1) component conditional distributions are Gaussian, 2) observations 
are independent given the mixture parameters, and 3) the number of components K (or a way to select it) is known or chosen. 

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

In [ ]:
Answer:
Spectral clustering assumes that the cluster structure of the data can be captured by a graph of pairwise 
similarities. In other words, points in the same cluster have high mutual affinity while points in different clusters have low affinity. 
By building an affinity matrix and using the leading eigenvectors of the graph Laplacian, spectral clustering 
assumes these eigenvectors reveal a low-dimensional embedding in which clusters become approximately linearly separable. 
This helps us by converting a potentially highly non-convex partitioning problem into a simpler clustering problem in 
eigenvector space — allowing recovery of non-spherical, manifold or connected shapes that k-means struggles with.

## Question 7
#### Define the gap statistic method. What do we use it for?

In [ ]:
Answer:
The gap statistic method estimates the number of clusters K. For each candidate K it 
compares the clustering dispersion observed in the data to the expected dispersion under a null reference distribution with no cluster structure. 
The method uses Gap(K) = E_null[log W(K)] − log W(K), where the null expectation is approximated by Monte Carlo sampling. 
We pick the K that gives a large gap. The gap statistic therefore tells us how much better the observed clustering is than random structure and is 
used to choose a principled number of clusters.